## Explore SMAP Dataset

In [1]:
import os 

import numpy as np
import pandas as pd

In [2]:
dataset_folder = '../data/SMAP_MSL'
file = os.path.join(dataset_folder, 'labeled_anomalies.csv')

values = pd.read_csv(file) 
values.head()

,chan_id,spacecraft,anomaly_sequences,class,num_values
0,P-1,SMAP,"[[2149, 2349], [4536, 4844], [3539, 3779]]","[contextual, contextual, contextual]",8505
1,S-1,SMAP,"[[5300, 5747]]",[point],7331
2,E-1,SMAP,"[[5000, 5030], [5610, 6086]]","[contextual, contextual]",8516
3,E-2,SMAP,"[[5598, 6995]]",[point],8532
4,E-3,SMAP,"[[5094, 8306]]",[point],8307


In [4]:
values.spacecraft.value_counts()

spacecraft
SMAP    55
MSL     27
Name: count, dtype: int64

In [5]:
values = values[values['spacecraft'] == "SMAP"]
# get the filename list from the chan_id
filenames = values['chan_id'].values.tolist()
filenames[:2]

['P-1', 'S-1']

In [7]:
def normalize3(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

In [10]:
for filename in filenames:
    # load the train and test numpy array values
    train = np.load(f'{dataset_folder}/train/{filename}.npy')
    test = np.load(f'{dataset_folder}/test/{filename}.npy')
    print("Train and test array shape : ", train.shape, test.shape)
    
    # normalize the values
    train, min_a, max_a = normalize3(train)
    test, _, _ = normalize3(test, min_a, max_a)

    # create the label array using the anomaly sequence
    labels = np.zeros(test.shape)
    indices = values[values['chan_id'] == filename]['anomaly_sequences'].values[0]
    indices = indices.replace(']', '').replace('[', '').split(', ')
    indices = [int(i) for i in indices]
    for i in range(0, len(indices), 2):
        labels[indices[i]: indices[i + 1], :] = 1

    break
    

Train and test array shape :  (2872, 25) (8505, 25)
